In [ ]:
import pandas as pd
import numpy as np

DATASET IMUNISASI

In [3]:
# read data untuk imunisasi
imun_bcg =  pd.read_csv("data.jabarprov.go.id/dinkes-od_17443_jml_imunisasi_bcg_pada_bayi__jk_data.csv")
imun_dasar =  pd.read_csv("data.jabarprov.go.id/dinkes-od_17464_jml_imunisasi_dasar_lengkap_pada_bayi__jk_data.csv")

data_bayi = pd.read_csv("data.jabarprov.go.id/dinkes-od_15942_jml_kelahiran_bayi__status_kelahiran_jk_data.csv")
data_bayi_hidup = data_bayi[data_bayi["status_kelahiran"] == "HIDUP"]
data_bayi_mati = data_bayi[data_bayi["status_kelahiran"] == "MATI"]

In [ ]:
# drop kolom yang tidak dibutuhkan
drop_kolom = ['id','kode_provinsi','nama_provinsi','satuan']

for kol in drop_kolom:
    data_bayi_hidup.drop([kol], axis=1, inplace=True)
    data_bayi_mati.drop([kol], axis=1, inplace=True)
    imun_bcg.drop([kol], axis=1, inplace=True)
    imun_dasar.drop([kol], axis=1, inplace=True)
    

In [6]:
# menggabungkan data dan mengubah kolom tahun ke datetime
data1 = pd.merge(data_bayi_hidup, imun_bcg, on=["kode_kabupaten_kota","nama_kabupaten_kota","tahun","jenis_kelamin"])
data2 = pd.merge(data1, imun_dasar, 
                 on=["kode_kabupaten_kota","nama_kabupaten_kota","tahun","jenis_kelamin"], 
                 suffixes=["_bcg","_dasar"])

data2.drop(["status_kelahiran"], axis=1, inplace=True)
data4 = pd.merge(data2, data_bayi_mati, 
                 on=["kode_kabupaten_kota","nama_kabupaten_kota","tahun","jenis_kelamin"], 
                 suffixes=["_hidup","_mati"])

data4.drop(["status_kelahiran"], axis=1, inplace=True)
data4["tahun"] = pd.to_datetime(data4["tahun"], format = "%Y")

In [ ]:
# export dataset imunisasi
data4.to_csv("imun_lahir_2017_2020.csv")

DATASET BBLR

In [12]:
# read data untuk BBLR
presentase_bayi_bblr = pd.read_csv("data.jabarprov.go.id/dinkes-od_17554_persentase_bayi_berat_badan_lahir_rendah_bblr__kabupat_data.csv")

In [15]:
# drop kolom
for kol in drop_kolom:
    presentase_bayi_bblr.drop([kol], axis=1, inplac

# grouping data
data_bayi_2 = data_bayi.groupby(["kode_kabupaten_kota","nama_kabupaten_kota","tahun"])["jumlah_kelahiran"].sum().reset_index()
data_bayi_2 = data_bayi_2[data_bayi_2["tahun"] >= 2019]
                              
# menggabungkan data untuk dengan jumlah kelahiran
data_bblr = pd.merge(presentase_bayi_bblr, data_bayi_2, how="outer",
                 on=["tahun","kode_kabupaten_kota","nama_kabupaten_kota"]
                )
data_bblr.fillna(0, inplace=True)

In [20]:
# membuat perhitungan kasar dengan menentukan jumlah bblr dari persentase dikali dengan total
jumlah_bblr = []

for i in range(len(data_bblr)):
    jumlah_bblr.append(data_bblr.iloc[i, 2]*data_bblr.iloc[i, 4]/100)

data_bblr["jumlah_bblr"] = jumlah_bblr

# mengubah kolom ke tipe data yang tepat untuk visualusasi
data_bblr['jumlah_kelahiran'] = data_bblr['jumlah_kelahiran'].map(int)
data_bblr['jumlah_bblr'] = data_bblr['jumlah_bblr'].map(int)
data_bblr["tahun"] = pd.to_datetime(data_bblr["tahun"], format = "%Y")

In [132]:
# export dataset jumlah bblr
data_bblr.to_csv("berat_bayi_lahir_rendah_2019_2020.csv")

DATASET IBU HAMIL

In [2]:
# read data
hamil_resti = pd.read_csv("data.jabarprov.go.id/dpmdes-idm_jml_ibu_hamil_mnglm_resiko_tinggi_kehamilan_resti__desa_kel_data.csv")
hamil_kronis = pd.read_csv("data.jabarprov.go.id/dpmdes-idm_jml_ibu_hamil_mnglm_kekurangan_energi_kronis_kek__desa_kelu_data.csv")
hamil = pd.read_csv("data.jabarprov.go.id/dinkes-od_17485_jumlah_ibu_hamil_berdasarkan_kabupatenkota_data.csv")

In [ ]:
# grouping data
hamil_kronis_kab = hamil_kronis.groupby(["bps_kode_kabupaten_kota","bps_nama_kabupaten_kota","tahun"])["jumlah_ibu_hamil_mengalami_kek"].sum().reset_index()
hamil_resti_kab = hamil_resti.groupby(["bps_kode_kabupaten_kota","bps_nama_kabupaten_kota","tahun"])["jumlah_ibu_hamil_resti"].sum().reset_index()
hamil = hamil.groupby(["kode_kabupaten_kota","nama_kabupaten_kota","tahun"])["jumlah_ibu_hamil"].sum().reset_index()

# mengambil data hamil untuk lebih dari 2017
hamil = hamil[hamil["tahun"] >= 2017]

# menggabungkan data
data_hamil_1 = pd.merge(hamil_kronis_kab, hamil_resti_kab, 
                 on=["bps_kode_kabupaten_kota","bps_nama_kabupaten_kota","tahun"],
                )
data_hamil_1.rename(columns = {'bps_kode_kabupaten_kota':'kode_kabupaten_kota', 'bps_nama_kabupaten_kota':'nama_kabupaten_kota'}, inplace = True)

data_hamil_2 = pd.merge(hamil, data_hamil_1, how="outer",
                 on=["tahun","kode_kabupaten_kota","nama_kabupaten_kota"]
                )

# mengubah kolom ke tipe data yang tepat untuk visualisasi
data4["jumlah_ibu_hamil"] = data4["jumlah_ibu_hamil"].map(int)
data4["jumlah_ibu_hamil_mengalami_kek"] = data4["jumlah_ibu_hamil_mengalami_kek"].map(int)
data4["jumlah_ibu_hamil_resti"] = data4["jumlah_ibu_hamil_resti"].map(int)
data4['kode_kabupaten_kota'] = data4['kode_kabupaten_kota'].map(int)
data_hamil_2["tahun"] = pd.to_datetime(hamil["tahun"], format = "%Y")


In [ ]:
# export dataset ke CSV
data_hamil_2.to_csv("status_ibu_hamil_2017_2020.csv")